# Top-level imports

In [78]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


#### import packages - setup

In [79]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import stats
from sklearn import metrics
import datetime
import tensorflow as tf

In [80]:
plt.style.use('fivethirtyeight')

In [81]:
data_dir = 'data/'

In [82]:
data_files = os.listdir(data_dir)
data_files.sort()
data_files

['BroughtonSeaLice_fishData.csv',
 'BroughtonSeaLice_fishInfo.csv',
 'BroughtonSeaLice_siteData.csv',
 'BroughtonSeaLice_siteInfo.csv',
 'DFOSeaLice_Data.csv',
 'DFOSeaLice_Info.csv',
 'IndustrySeaLice_Data.csv',
 'IndustrySeaLice_Info.csv',
 'README.md']

#### Wild data

In [83]:
fish_data = pd.read_csv(f'{data_dir}BroughtonSeaLice_fishData.csv', encoding='ISO-8859-1')
site_data = pd.read_csv(f'{data_dir}BroughtonSeaLice_siteData.csv', encoding='ISO-8859-1')

/usr/local/Cellar/ipython/7.8.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
fish_info = pd.read_csv(f'{data_dir}BroughtonSeaLice_fishInfo.csv', encoding='ISO-8859-1')
site_info = pd.read_csv(f'{data_dir}BroughtonSeaLice_siteInfo.csv', encoding='ISO-8859-1')

#### Farm data

In [85]:
dfo_data = pd.read_csv(f'{data_dir}DFOSeaLice_Data.csv')
dfo_info = pd.read_csv(f'{data_dir}DFOSeaLice_Info.csv')

In [86]:
industry_data = pd.read_csv(f'{data_dir}IndustrySeaLice_Data.csv', encoding='ISO-8859-1', low_memory=False)
industry_info = pd.read_csv(f'{data_dir}IndustrySeaLice_Info.csv', encoding='ISO-8859-1', low_memory=False)

##### Data/Feature Engineering

Possible input scenarios
- 2001-2018: We have to trust that the model can work with the large amounts of NaN values in earlier years, both in wild data and no farmed data until 2011 
- 2003-2018: 2003 is the first year we have data starting in March


## Setting overall constants

Things to set here 
- Years to analyse
- Within-season date range
- Accepted ranges
- Resampling dates

In [365]:
analysis_years = list(range(2003, 2018))

In [366]:
analysis_months = list(range(3, 8))

In [367]:
dow_dict = {
    1: 'MON',
    2: 'TUE',
    3: 'WED',
    4: 'THU', 
    5: 'FRI', 
    6: 'SAT', 
    7: 'SUN'
}

def get_dow(dt_obj):
    dow_text = dt_obj.isoweekday()
    return(dow_dict[dow_text])

## Setting up response

### Unified adult count
This is one possible response

In [402]:
adult = fish_data[['Lep_PAmale', 'Lep_PAfemale', 
                   'Lep_male', 'Lep_gravid',
                   'Lep_nongravid', 'unid_PA',
                   'unid_adult']].sum(axis=1)

In [403]:
fish_data_date = pd.to_datetime(fish_data[['year', 'day', 'month']])

In [404]:
response = pd.DataFrame({'count':adult.values, 
                         'location':fish_data['location'].values,
                         'datetime': fish_data_date})

In [405]:
response_glacier = response[response['location'] == 'Glacier']

In [406]:
year_df_list = []
for year in analysis_years:
    subset = response_glacier[response_glacier['datetime'].dt.year == year]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.loc[1] = np.nan
    subset.loc[1, 'datetime'] = datetime.datetime(year, 12, 31)
    subset.sort_values('datetime', inplace=True)
    subset_resampled = subset.resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                       on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_df_list.append(subset_resampled)
Y_glacier = pd.concat(year_df_list)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/l

In [407]:
Y_glacier

,count
datetime,
2002-12-25,NaN
2003-01-01,NaN
2003-01-08,NaN
2003-01-15,NaN
2003-01-22,NaN
2003-01-29,NaN
2003-02-05,NaN
2003-02-12,NaN
2003-02-19,NaN


resample by week for average

This is the code I was formerly using for the resampling, it's pretty inconsistent

Y_glacier = response_glacier.resample('W', on='datetime', label='left').apply(np.nanmean)

## Setting up inputs - wild data

## Setting up inputs - wild data

In [408]:
year_df_list = []
for year in analysis_years:
    subset = response_glacier[response_glacier['datetime'].dt.year == year]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.loc[1] = np.nan
    subset.loc[1, 'datetime'] = datetime.datetime(year, 12, 31)
    subset.sort_values('datetime', inplace=True)
    subset = subset.resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}', 
                             on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_df_list.append(subset[subset.index.month.isin(analysis_months)])
Y_glacier = pd.concat(year_df_list)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/l

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

In [409]:
# non-motile lice
juvenile = pd.DataFrame(fish_data[['Lep_cope', 'chalA',
                      'chalB', 'Caligus_cope',
                      'unid_cope', 'chal_unid']].sum(axis=1)).rename({0: 'count'}, axis=1)
juvenile['datetime'] = fish_data_date
juvenile['location'] = fish_data['location']

In [410]:
X_wild_juv

count
location datetime            
Burdwood 2003-03-12  0.000000
         2003-03-19  0.761905
         2003-03-26       NaN
         2003-04-02  0.175000
         2003-04-09  0.363636
         2003-04-16  0.368421
         2003-04-23  0.650000
         2003-04-30  0.722222
         2003-05-07       NaN
         2003-05-14  0.540541
         2003-05-21       NaN
         2003-05-28  0.195122
Glacier  2003-02-26  0.000000
         2003-03-05       NaN
         2003-03-12       NaN
         2003-03-19  0.285714
         2003-03-26       NaN
         2003-04-02  0.150000
         2003-04-09  0.368421
         2003-04-16  0.100000
         2003-04-23  0.350000
         2003-04-30  1.000000
         2003-05-07       NaN
         2003-05-14  1.357143
         2003-05-21       NaN
         2003-05-28  0.380952
Wicklow  2003-03-12  1.000000
         2003-03-19       NaN
         2003-03-26  0.210526
         2003-04-02  0.741935
...                       ...
Burdwood 2017-05-07  0.920000
         2017-05-14  0.800000
         2017-05-21  0.880000
         2017-05-28  0.970000
         2017-06-04  0.227273
         2017-06-11       NaN
         2017-06-18       NaN
         2017-06-25  0.000000
Glacier  2017-04-16  0.290909
         2017-04-23  0.518519
         2017-04-30  0.611765
         2017-05-07  2.400000
         2017-05-14       NaN
         2017-05-21  1.200000
         2017-05-28  2.653465
         2017-06-04  0.461538
         2017-06-11       NaN
         2017-06-18  0.352941
Wicklow  2017-04-09  0.430000
         2017-04-16  0.640000
         2017-04-23       NaN
         2017-04-30  1.500000
         2017-05-07  2.923077
         2017-05-14  2.216867
         2017-05-21  2.013514
         2017-05-28  2.930556
         2017-06-04  0.462687
         2017-06-11       NaN
         2017-06-18  1.654206
         2017-06-25  1.063492

[551 rows x 1 columns]

In [411]:
year_juv_list = []
for year in analysis_years:
    subset = juvenile[juvenile['datetime'].dt.year == year]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('location').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                          on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_juv_list.append(subset_resample)
X_wild_juv = pd.concat(year_juv_list)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.htm

This is the code I was previously using to resample the juvenile counts, it's pretty inconsistent 


X_wild_juv = juvenile.groupby('location').resample('W', on='datetime', label='left').apply(np.nanmean)

In [412]:
site_data['datetime'] = pd.to_datetime(site_data[['year', 'month', 'day']])

In [468]:
year_temp_list = []
for year in analysis_years:
    subset = site_data.loc[(site_data['datetime'].dt.year == year), ['datetime', 'temp', 'location']]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.loc[1] = np.nan
    subset.loc[1, 'datetime'] = datetime.datetime(year, 12, 31)
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('location').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                          on='datetime', label='left').apply(np.nanmean).interpolate(methods='linear')
    year_temp_list.append(subset_resample)
X_wild_temp = pd.concat(year_temp_list)

This is how I used to get these inputs, now outdated

X_wild_temp = site_data[['datetime', 'temp', 'location']]
X_wild_temp = X_wild_temp.groupby('location').resample('w', on='datetime', label='left').apply(np.nanmean)

In [445]:
year_sal_list = []
for year in analysis_years:
    subset = site_data.loc[(site_data['datetime'].dt.year == year), ['datetime', 'salt', 'location']]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.loc[1] = np.nan
    subset.loc[1, 'datetime'] = datetime.datetime(year, 12, 31)
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('location').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                          on='datetime', label='left').apply(np.nanmean).interpolate(method='linear')
    year_sal_list.append(subset_resample)
X_wild_sal = pd.concat(year_sal_list)

How I used to get these inputs, now outdated

X_wild_sal = site_data[['datetime', 'salt', 'location']]
X_wild_sal = X_wild_sal.groupby('location').resample('w', on='datetime', label='left').apply(np.nanmean)

## Setting up inputs - farm data

- Implementing treatments *may* not be a priority - I'll have to do some extensive testing on this
- There are of course concerns with using the industry counts as they *may* be of very low quality (I need to look much more extensively into this data)
    - Should look into what kind of predictive differences including/excluding these has 

Inputs that we want

44 - Sargeunts Pass

41 - Doctor Islet

45 - Humphrey Rock

56 - Glacier Falls

54 - Simoom Sound

50 - Burdwood Islands

53 - Sir Edmond Bay

49 - Wicklow Point 

In [415]:
industry_data[industry_data['Site Common Name'].str.contains('Sir')]['Site Common Name'].unique()
# Can't find Simoom Sound??

array(['Sir Edmund Bay'], dtype=object)

In [416]:
relevant_farms = ['Sargeaunt Pass',
                  'Doctor Islet',
                  'Humphrey Rock',
                  'Simoom Sound*',
                  'Burdwood',
                  'Glacier Falls',
                  'Sir Edmund Bay',
                  'Wicklow Point'
                 ]

In [417]:
industry_data[industry_data['Site Common Name'].str.contains('|'.join(relevant_farms))]['Site Common Name'].unique()

array(['Burdwood', 'Sir Edmund Bay', 'Doctor Islets', 'Glacier Falls',
       'Wicklow Point', 'Sargeaunt Pass', 'Humphrey Rock',
       'Wicklow Point - post treatment', 'Wicklow Point - pre treatment',
       'Wicklow Point - post-treatment', 'Wicklow Point - pre-treatment'],
      dtype=object)

From what I can tell - most of these farms don't actually have a treatment indicator

Wondering if this is significant? 

I also can't find Simoom Sound, maybe it's here under a different name? 

In [418]:
comments_examples = industry_data[industry_data['Site Common Name'] == 'Burdwood']['Comments'].unique()

This is how I was resampling the industry data before

X_industry = relevant_farm_data[['datetime',
                            'Site Common Name',
                            'Average L. salmonis motiles per fish',
                            'Average chalimus per fish']]
X_industry = X_industry.groupby('Site Common Name').resample('W', on='datetime', label='left').mean()

In [454]:
relevant_farm_data = industry_data[industry_data['Site Common Name'].str.contains('|'.join(relevant_farms))]

relevant_farm_data['Day'] = 1
month_map = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
relevant_farm_data['month'] = relevant_farm_data['Month'].map(month_map)
relevant_farm_data['datetime'] = pd.to_datetime(relevant_farm_data[['Year', 'month', 'Day']])

relevant_farm_data = relevant_farm_data[relevant_farm_data['datetime'].dt.year.isin(analysis_years)]

year_industry_list = []
for year in analysis_years:
    subset = relevant_farm_data.loc[(relevant_farm_data['datetime'].dt.year == year), 
                                   ['datetime', 'Site Common Name', 'Average L. salmonis motiles per fish']]
    subset.loc[0] = np.nan
    subset.loc[0, 'datetime'] = datetime.datetime(year, 1, 1)
    subset.loc[1] = np.nan
    subset.loc[1, 'datetime'] = datetime.datetime(year, 12, 31)
            
    subset.sort_values('datetime', inplace=True)
    subset_resample = subset.groupby('Site Common Name').resample(f'W-{get_dow(datetime.datetime(year, 1, 1))}',
                                                                 on='datetime', label='left').apply(np.nanmean).interpolate(method='linear')
    year_industry_list.append(subset_resample)
X_industry = pd.concat(year_industry_list)


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

### Forming array for model input

In [427]:
arrays_to_stack = []

for year in analysis_years:
    juv_sub = X_wild_juv[(X_wild_juv.index.get_level_values(1).year == year) & 
                         X_wild_juv.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values
    temp_sub = X_wild_temp[(X_wild_temp.index.get_level_values(1).year == year) & 
                           X_wild_temp.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values
    sal_sub = X_wild_sal[(X_wild_sal.index.get_level_values(1).year == year) & 
                         X_wild_sal.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values
    ind_sub = X_industry[(X_industry.index.get_level_values(1).year == year) & 
                         X_industry.index.get_level_values(1).month.isin(analysis_months)].unstack().T.values
    
    year_array = np.concatenate((juv_sub, temp_sub, sal_sub, ind_sub), axis=1)
#     if year_array.shape[0] == 21:
#         year_array = np.append(year_array, np.full(9, np.nan).reshape(1, 9), axis=0)
    arrays_to_stack.append(year_array)
    
test_X = np.stack(arrays_to_stack, axis=0)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
test_X.shape

In [432]:
arrays_to_stack = []

for year in analysis_years:
    year_Y = Y_glacier[(Y_glacier.index.year == year) & 
                       Y_glacier.index.month.isin(analysis_months)].values
#     if year_Y.shape[0] == 21:
#         year_Y = np.append(year_Y, np.nan)
#         year_Y = year_Y.reshape(22, 1)
    arrays_to_stack.append(year_Y)
    
test_Y = np.stack(arrays_to_stack, axis=0)
test_Y = test_Y

In [433]:
test_Y.shape

(15, 22, 1)

# Experimenting with models

## Import packages and set up functions

Write normalising functions here (let's go with MinMaxScaling for future implementations)

## Setup Data

### Normalise and fill NAs

This section is pretty bad!

This is an issue being worked on, I need to:
* Render the data stationary
* Use a different scaling function
* As a QOL update - right now these cells matter heavily on the proper order of cells being executed (top to bottom) in the notebook, that's pretty bad and should be fixed

In [112]:
X_mean = np.nanmean(test_X, axis=1)
for i in range(0, X_mean.shape[0]):
    test_X[[i]] = test_X[[i]] - X_mean[[i]]
X_std = np.nanstd(test_X, axis=1)
for i in range(0, X_std.shape[0]):
    test_X[[i]] = test_X[[i]] - X_std[[i]]
test_X = np.nan_to_num(test_X)

In [113]:
Y_mean = np.nanmean(test_Y, axis=1)
for i in range(0, Y_mean.shape[0]):
    test_Y[[i]] = test_Y[[i]] - Y_mean[[i]]
Y_std = np.nanstd(test_Y, axis=1)
for i in range(0, Y_std.shape[0]):
    test_Y[[i]] = test_Y[[i]] - Y_std[[i]]
test_Y = np.nan_to_num(test_Y)

### Split into train and test

In [114]:
train_X = test_X[:-3]
train_Y = test_Y[:-3]

eval_X = test_X[-3:]
eval_Y = test_Y[-3:]

## Baseline

### Overall Avg

### Naive seasonal

In [115]:
def naive_seasonal(year: int, reference_data: pd.DataFrame):
    '''
    Naive seasonal model
    This model takes a year to be predicted in and returns the last known year's values
    
    year: Year to be predicted
    reference_data: Pandas dataframe of the test/input Y data, must have a DatetimeIndex index
    '''
    
    
    pred_subset = reference_data[reference_data.index.year == year]
    preds = pred_subset[pred_subset.index.month.isin(analysis_months)]
    
    return(np.array(preds))

### ARIMA

## Non-parametric

Some high level notes
- This analysis is very rough and is in no way final!!!
- The normalising, feature engineering, etc. is probably the roughest part of all of this. I don't expect it to have an effect on model choice but by no means should the input/output data be taken verbatim as what I intend to use
- Optimizers: some reading has shown that RMSprop is the suggested optimiser for RNNs, this also coincides with François Chollet's use of optimisers so for now I am going with this one for RNNs and ADAM as the default for all others. This requires further research!
- Reference points: 
     - ARIMA
     - Mean & SD

### Setting up tensorboard logs

### LSTM

Long short-term memory (LSTM)

This model is the most advanced RNN for sequential data, so the highest potential upside in gaining value from features but may also be overkill so make sure to compare it to some other RNN baselines

In [116]:
!rm -rf ./logs/ 
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [117]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(100, input_shape=(22, 9),
               return_sequences=True,
               activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(100, input_shape=(22, 9), 
               return_sequences=True,
               activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(100, input_shape=(22, 9),
               return_sequences=True,
               activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='rmsprop', 
              loss='mse',
              metrics=['acc'])

history = model.fit(train_X, train_Y, 
                    epochs=100, batch_size=16,
                    validation_split=0.2,
                    callbacks=[tensorboard_callback])

Train on 9 samples, validate on 3 samples
Epoch 1/100
9/9 [==============================] - 4s 417ms/sample - loss: 0.5367 - acc: 0.4848 - val_loss: 0.0253 - val_acc: 0.4394
Epoch 2/100
9/9 [==============================] - 1s 68ms/sample - loss: 0.4839 - acc: 0.4848 - val_loss: 0.0126 - val_acc: 0.4394
Epoch 3/100
9/9 [==============================] - 0s 8ms/sample - loss: 0.4665 - acc: 0.4596 - val_loss: 0.0095 - val_acc: 0.4394
Epoch 4/100
9/9 [==============================] - 0s 8ms/sample - loss: 0.3809 - acc: 0.4848 - val_loss: 0.0045 - val_acc: 0.4394
Epoch 5/100
9/9 [==============================] - 0s 8ms/sample - loss: 0.3467 - acc: 0.4848 - val_loss: 0.0086 - val_acc: 0.4394
Epoch 6/100
9/9 [==============================] - 0s 7ms/sample - loss: 0.3270 - acc: 0.4545 - val_loss: 0.0036 - val_acc: 0.4394
Epoch 7/100
9/9 [==============================] - 0s 7ms/sample - loss: 0.3603 - acc: 0.4848 - val_loss: 0.0041 - val_acc: 0.4394
Epoch 8/100
9/9 [=====================

Epoch 62/100
9/9 [==============================] - 0s 8ms/sample - loss: 0.0474 - acc: 0.4798 - val_loss: 0.0060 - val_acc: 0.4394
Epoch 63/100
9/9 [==============================] - 0s 6ms/sample - loss: 0.0723 - acc: 0.4848 - val_loss: 0.0056 - val_acc: 0.4394
Epoch 64/100
9/9 [==============================] - 0s 7ms/sample - loss: 0.0604 - acc: 0.4848 - val_loss: 0.0055 - val_acc: 0.4394
Epoch 65/100
9/9 [==============================] - 0s 6ms/sample - loss: 0.0750 - acc: 0.4848 - val_loss: 0.0037 - val_acc: 0.4394
Epoch 66/100
9/9 [==============================] - 0s 7ms/sample - loss: 0.0694 - acc: 0.4848 - val_loss: 0.0040 - val_acc: 0.4394
Epoch 67/100
9/9 [==============================] - 0s 6ms/sample - loss: 0.0494 - acc: 0.4798 - val_loss: 0.0058 - val_acc: 0.4394
Epoch 68/100
9/9 [==============================] - 0s 7ms/sample - loss: 0.0909 - acc: 0.4848 - val_loss: 0.0032 - val_acc: 0.4394
Epoch 69/100
9/9 [==============================] - 0s 6ms/sample - loss: 0.

In [118]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 52918), started 0:54:50 ago. (Use '!kill 52918' to kill it.)

In [119]:
train_predictions = model.predict(train_X)

# Train RMSE
metrics.mean_squared_error(train_Y.reshape(train_Y.shape[0], train_Y.shape[1] * train_Y.shape[2]),
                           train_predictions.reshape(train_predictions.shape[0], train_predictions.shape[1] * train_predictions.shape[2]))

0.011715250695256259

In [120]:
test_predictions = model.predict(eval_X)

# Test RMSE
metrics.mean_squared_error(eval_Y.reshape(eval_Y.shape[0], eval_Y.shape[1] * eval_Y.shape[2]),
                           test_predictions.reshape(test_predictions.shape[0], test_predictions.shape[1] * train_predictions.shape[2]))

0.056253281305406744

### CNN

1D Convnet

1D convnets have sometimes shown to be more efficient with data than similarly sized RNNs when dealing with small problems. Maybe that means they're a good application here? Each output timestep takes power from its neighbours, so this is assuming some sort of autoregression.

In [121]:
from keras.models import Sequential
from keras import layers

In [122]:
train_Y.shape

(12, 22, 1)

In [123]:
model = Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=2,
                        activation='relu',
                        input_shape=(22,6)))
model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1))
# 1D Convnets cannot return sequences, they converge to one value...
model.compile(optimizer='adam', 
              loss='mse',
              metrics=['acc'])

history = model.fit(train_X, train_Y.reshape(12, 22), 
                    epochs=100, batch_size=16,
                    validation_split=0.2)

ValueError: Error when checking input: expected conv1d_2_input to have shape (22, 6) but got array with shape (22, 9)

## Parametric

### ARIMA